In [ ]:
# 3_train_model

# compiled data
# process data for machine learning
# train model
# plot loss and metrics
# save trained model

In [ ]:
# import libraries

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import ReLU, Softmax
from tensorflow.keras.activations import *
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam

from plotly import graph_objects as go

2023-04-04 17:47:48.933419: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# load data

# directory were compiled data was saved
path_from = "/path_of_directory_where_compiled data_was_saved/"

# load data
data = np.load(path_from + "data.npy")
labels = np.load(path_from + "labels.npy")

In [8]:
# reshape data to get a 3d array and 1 label per video

data = data.reshape(int(data.shape[0]/50), 50, 1659)
labels = labels[::50]

In [9]:
# process data for machine learning

# Split 80% in train 10% in val 10% in test
X_train, X_valtest, y_train, y_valtest = train_test_split(data,
                                                          labels,
                                                          test_size=0.2,
                                                          stratify=labels)
X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest,
                                                test_size=0.5,
                                                stratify=y_valtest)

# encode labels
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_val = encoder.transform(y_val)
y_test = encoder.transform(y_test)

In [10]:
# make tensors

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [11]:
# shuffle and batch

train_ds = train_ds.shuffle(data.shape[0]).batch(512)
val_ds = val_ds.batch(512)
test_ds = test_ds.batch(512)

In [15]:
# create model

model = Sequential([
    Masking(mask_value = 0),
    GRU(128, return_sequences = True),
    Dropout(0.2),
    Dense(128, "relu"),
    GRU(64, return_sequences = True),
    Dropout(0.2),
    Dense(64, "relu"),
    GRU(32, return_sequences = False),
    BatchNormalization(),
    Dense(32, "relu"),
    Dense(len(np.unique(labels)), "softmax")
])

In [16]:
# compile model

ed = ExponentialDecay(
    initial_learning_rate = 0.0008, 
    decay_steps = 10, 
    decay_rate = 0.98)

optimizer = Adam(learning_rate = ed)

model.compile(
    optimizer,
    loss = SparseCategoricalCrossentropy(),
    metrics = [SparseCategoricalAccuracy()])

In [17]:
# train model

history = model.fit(train_ds, epochs = 50, validation_data = val_ds)

Epoch 1/100
161/161 [==============================] - 78s 294ms/step - loss: 2.1067 - sparse_categorical_accuracy: 0.2633 - val_loss: 2.0599 - val_sparse_categorical_accuracy: 0.2712
Epoch 2/100
161/161 [==============================] - 61s 348ms/step - loss: 1.8999 - sparse_categorical_accuracy: 0.3441 - val_loss: 1.8342 - val_sparse_categorical_accuracy: 0.3702
Epoch 3/100
161/161 [==============================] - 79s 415ms/step - loss: 1.7506 - sparse_categorical_accuracy: 0.4026 - val_loss: 2.0874 - val_sparse_categorical_accuracy: 0.2814
Epoch 4/100
161/161 [==============================] - 77s 442ms/step - loss: 1.5839 - sparse_categorical_accuracy: 0.4685 - val_loss: 1.6062 - val_sparse_categorical_accuracy: 0.4645
Epoch 5/100
161/161 [==============================] - 75s 425ms/step - loss: 1.4784 - sparse_categorical_accuracy: 0.5063 - val_loss: 2.0068 - val_sparse_categorical_accuracy: 0.3905
Epoch 6/100
161/161 [==============================] - 78s 408ms/step - loss: 1.

In [ ]:
# save model

model.save("/path_were_the_model_should_be_saved/model_date_epochnumber.h5")

In [18]:
# set color chart

color_chart = ["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]

In [19]:
# plot loss

fig = go.Figure(data=[
                      go.Scatter(
                          y=history.history["loss"],
                          name="Training loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[0]
                          )),
                      go.Scatter(
                          y=history.history["val_loss"],
                          name="Validation loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[1]
                          ))
])
fig.update_layout(
    title='Training and val loss across epochs',
    xaxis_title='epochs',
    yaxis_title='Cross Entropy'    
)
fig.show()

In [20]:
# plot accuracy

fig = go.Figure(data=[
                      go.Scatter(
                          y=history.history["sparse_categorical_accuracy"],
                          name="Training accuracy",
                          mode="lines",
                          marker=dict(
                              color=color_chart[4]
                          )),
                      go.Scatter(
                          y=history.history["val_sparse_categorical_accuracy"],
                          name="Validation accruracy",
                          mode="lines",
                          marker=dict(
                              color=color_chart[5]
                          ))
])
fig.update_layout(
    title='Training and val accuracy across epochs',
    xaxis_title='epochs',
    yaxis_title='Accuracy'    
)
fig.show()